# MMM Algorithms and Optimization

In this hands-on you will implement and analyze matrix-matrix multiplication (MMM) algorithms. 

**Goals**:

*   Undestand the role of cache in MMM optimization.
*   Implement naive, recursive and Strassen algorithms and compare their performance.
*   Measure MMM execution times achieved by different frameworks on different hardware.





Let’s refresh our mind about matrix multiplication:

<img src="https://miro.medium.com/max/1400/1*HCKLKLOm5clIeG-d0cRLhA.gif" alt="MMM" style="width: 150px;"/>

It consists of multiplication and addition, this naive way has cubic complexity. For example, the complexity of the 4x4 matrix multiplication is $O(4^3)$ while 10x10 matrix multiplication is $O(10^3)$.

Explore the characteristics of the machine used to run your code. Thanks, Google!


In [ ]:
!lscpu

Import several helpful packages.

In [ ]:
import numpy as np
import tensorflow as tf

import timeit
from tqdm import tqdm
from functools import partial
import matplotlib.pyplot as plt

## Row-major and column-major

Dense data are stored contiguously in memory, addressed by a single index (the memory address). Array memory ordering schemes translate that single index into multiple indices corresponding to the array coordinates. For example, matrices have two indices: rows and columns. Three-d arrays have three, and so on.

*Column-major order* is used by **Fortran, Matlab, R**, and most underlying core linear algebra libraries (BLAS). Sequential address locations are translated into array coordinates $i$, $j$, $k$, … so that the first array coordinates vary most rapidly with address, the next array coordinates less rapidly, and so on. For instance, four address locations $1$, $2$, $3$, $4$ are translated into a two by two matrix coordinate system $(i, j)$ as:

```
address   i  j
  1       1  1
  2       2  1
  3       1  2
  4       2  2
```
*Row-major* ordering is a different translation between sequential address indices and array coordinates, instead laying sequential data in memory across rows in matrices. Row-major ordering is sometimes called **C/C++** style ordering. For instance address locations 1, 2, 3, 4 are translated into a 2x2 matrix coordinate system (i, j) as:

```
address   i  j
  1       1  1
  2       1  2
  3       2  1
  4       2  2
```

**Exercise**: How are matrices stored in memory in Python / with NumPy? Row-major or column major?

In [ ]:
### START CODE HERE ### (≈ 2 lines of code)
pass
### END CODE HERE ###

## Cache and Loop Ordering

**Exercise**: Implement naive MMM algorithm for computing $A^2$ of matrix $A$ at O(n^3) runtime.

In [ ]:
# Naive matrix multiplication: b = a * a, a: NxN and b: NxN.
# Running time: O(N^3)
def naive_mmm(A):
  B = np.zeros(A.shape)
  ### START CODE HERE ### (≈ 6 lines of code)
  pass
  ### END CODE HERE ###
  return B

Test your implementation.

In [ ]:
a = np.array([[1, 2, 3, 4], [4, 5, 6, 7], [7, 8, 9, 1], [2, 3, 4, 5]])
print(naive_mmm(a))

We will use a simple and lightweight cache implementaion `@lru_cache` to keep track of repeatable access to the same elements of a matrix. The cache is fully associative cache with Least Recently Used (LRU) replacement policy.

We fix the cache size `maxsize=32`, although you can experiment also with a different cache size.

In [ ]:
from functools import lru_cache

N = 16
A = np.random.randn(N,N)
B = np.zeros(A.shape)

# Fully associative cache. Block size=1. Temporal locality only.
@lru_cache(maxsize=32)
def access(i,j):
  return A[i,j]

# Naive matrix multiplication: b = a * a, a: NxN and b: NxN.
# Running time: O(N^3)
def naive_mmm_cache(n):
  ### START CODE HERE ### (≈ 4 lines of code, re-use your previous solution)
  pass
  ### END CODE HERE ###

access.cache_clear()
naive_mmm_cache(N)
access.cache_info()

**Exercise**: How does the loop order impact cache performance and why?

Let's measure the number of hits and misses for different matrix sizes: [16, 32, ..., 128].

In [ ]:
all_hits = np.array([])
all_misses = np.array([])

N_max = 128
A = np.random.randn(N_max,N_max)

Z = range(16, N_max, 1)
for n in tqdm(Z):
  B = np.zeros(A.shape)
  access.cache_clear()    # clear cache
  naive_mmm_cache(n)      # compute
  x = access.cache_info() # get cache statistics
  all_hits = np.append(all_hits, x.hits)
  all_misses = np.append(all_misses, x.misses)

Plot `all_hits` and `all_misses` for multiplying matrices of different sizes. 

**Exercise**: Re-compute the plot for different orderings of $i$, $j$ and $k$ loops in the matrix computation routine above. Explain the differences!

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(Z, all_hits, 'r-')
plt.plot(Z, all_misses, 'b-')
plt.yscale('log')
plt.xlabel('Matrix size')
plt.ylabel('Hits / Misses')
plt.legend(['Hits', 'Misses'])
plt.show()

## Recursive Matrix Multiplication

<img src="https://www.geeksforgeeks.org/wp-content/uploads/strassen_new.png">

In [ ]:
def split(matrix):
	"""
	Splits a given matrix into quarters.
	Input:  NxN matrix
	Output: Tuple containing 4 N/2 x N/2 matrices corresponding to a, b, c, d
	"""
  
  ### START CODE HERE ### (≈ 3 lines of code)
	return None
  ### END CODE HERE ###

def recursive(x, y):
	"""
	Computes matrix product by divide and conquer approach, recursively.
	Input:  NxN matrices x and y
	Output: NxN matrix, product of x and y
	"""

	# Base case when size of matrices is 1x1
	if len(x) == 1:
		return x * y

	# Splitting the matrices into quadrants. This will be done recursively
	# until the base case is reached.
  ### START CODE HERE ### (≈ 2 lines of code)
	None
  ### END CODE HERE ###

	# Computing the values of the 4 quadrants of the final matrix c
  ### START CODE HERE ### (≈ 4 lines of code)
	c11 = None
	c12 = None
	c21 = None
	c22 = None
  ### END CODE HERE ###

	# Combining the 4 quadrants into a single matrix by stacking horizontally and vertically.
	c = np.vstack((np.hstack((c11, c12)), np.hstack((c21, c22))))
	return c

In [ ]:
A = np.array([[1, 2, 3, 4], [4, 5, 6, 7], [7, 8, 9, 1], [2, 3, 4, 5]])
B = np.zeros(A.shape)
print(recursive(A,A))

## Strassen Algorithm

<img src="https://www.geeksforgeeks.org/wp-content/uploads/stressen_formula_new_new.png">

Hint: Use the same `split` function from before.

In [ ]:
def strassen(x, y):
	"""
	Computes matrix product by divide and conquer approach, recursively.
	Input: nxn matrices x and y
	Output: nxn matrix, product of x and y
	"""

	# Base case when size of matrices is 1x1
	if len(x) == 1:
		return x * y

	# Splitting the matrices into quadrants. This will be done recursively
	# until the base case is reached.
  ### START CODE HERE ### (≈ 2 lines of code)
	None
  ### END CODE HERE ###

	# Computing the 7 products, recursively (p1, p2...p7)
  ### START CODE HERE ### (≈ 7 lines of code)
	None
  ### END CODE HERE ###

	# Computing the values of the 4 quadrants of the final matrix c
  ### START CODE HERE ### (≈ 4 lines of code)
	c11 = None
	c12 = None
	c21 = None
	c22 = None
  ### END CODE HERE ###

	# Combining the 4 quadrants into a single matrix by stacking horizontally and vertically.
	c = np.vstack((np.hstack((c11, c12)), np.hstack((c21, c22))))
	return c

Test your implementation.

In [ ]:
A = np.array([[1, 2, 3, 4], [4, 5, 6, 7], [7, 8, 9, 1], [2, 3, 4, 5]])
B = np.zeros(A.shape)
print(strassen(A,A))

Measure timing with `%timeit` magic command. The number `-n` and repeat `-r` serve different purposes. The *number* controls how many executions are done for each timing and it's used to get representative timings. The *repeat* argument controls how many timings are done and its use is to get accurate statistics.

In [ ]:
N=32
A = np.random.randn(N,N)
B = np.zeros(A.shape)

%timeit -n 10 -r 10 naive_mmm(A)
%timeit -n 10 -r 10 recursive(A,A)
%timeit -n 10 -r 10 strassen(A,A)

## MMM on CPU / GPU with Different Frameworks

Change your google colab runtime to using GPU.

The NVIDIA System Management Interface (`nvidia-smi` utility) is intended to aid in the management and monitoring of NVIDIA GPU devices.

In [ ]:
!nvidia-smi

P0 is the highest performance/power state, P15 is the lowest performance/power state.

MiB = Mebibyte
GiB = Gibibyte

GB vs GiB: One GB is defined as 1000³ (1,000,000,000) bytes and one GiB as 1024³ (1,073,741,824) bytes. That means one GB equals 0.93 GiB.

Since we changed the runtime, we need to imports all required packages again.

In [ ]:
import numpy as np 
import torch

import timeit
from tqdm import tqdm
from functools import partial
import matplotlib.pyplot as plt

The functions below implement MMM using optimized primitives provided by different frameworks.

NumPy implementation:

In [ ]:
def np_mmm(N):
  a = np.random.randn(N,N)
  b = np.matmul(a,a)
  del a, b

TensorFlow implementation. CPU and GPU implementations are provided. 

By default, tensors are created on the CPU. It is important to note that whenever we want to operate on multiple terms, they need to be on the same device. For instance, if we sum two tensors, we need to make sure that both arguments live on the same device---otherwise the framework would not know where to store the result or even how to decide where to perform the computation.

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def tf_mmm(N):
  with tf.device('/cpu:0'):
    a = tf.random.normal((N, N))
    b = tf.math.multiply(a,a)
  del a, b

def tf_mmm_gpu(N):
  with tf.device('/device:GPU:0'):
    a = tf.random.normal((N, N))
    b = tf.math.multiply(a,a)
  del a, b

PyTorch. Make sure the device is set to GPU. CPU and GPU implementations of MMM with PyTorch are provided.

In [ ]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


In [ ]:
def torch_mmm(N):
  a = torch.randn(N,N)
  b = torch.matmul(a,a)
  del a, b

def torch_mmm_cuda(N):
  a = torch.randn(N,N).to(device)
  b = torch.matmul(a,a)
  del a, b 

Timing and plotting routines.

In [ ]:
def plot_time(func, inputs, repeats, n_tests):
    """
    Run timer and plot time complexity of `func` using the iterable `inputs`.
    Run the function `n_tests` times per `repeats`.
    """
    x, y, yerr = [], [], []
    for i in (inputs):
        timer = timeit.Timer(partial(func, i))
        t = timer.repeat(repeat=repeats, number=n_tests)
        x.append(i)
        y.append(np.mean(t))
        yerr.append(np.std(t) / np.sqrt(len(t)))
    plt.errorbar(x, y, yerr=yerr, fmt='-o', label=func.__name__)


def plot_times(functions, inputs, repeats=3, n_tests=1, file_name=""):
    """
    Run timer and plot time complexity of all `functions`, using the iterable `inputs`.
    Run the functions `n_tests` times per `repeats`.
    Adds a legend containing the labels added by `plot_time`.
    """
    for func in tqdm(functions):
        plot_time(func, inputs, repeats, n_tests)
    plt.legend()
    plt.xlabel("Input")
    plt.ylabel("Time [s]")
    plt.yscale("log")
    if not file_name:
        plt.show()
    else:
        plt.savefig(file_name)

The `%time` magic command measures the execution time of the line which follows it using the time python module. `%%time` measures the execution time of the whole cell.

*Wall clock time* is the actual amount of time taken to perform a job. This is equivalent to timing your job with a stopwatch and the measured time to complete your task can be affected by anything else that the system happens to be doing at the time.

*User time* measures the amount of time the CPU spent running your code. This does not count anything else that might be running, and also does not count CPU time spent in the kernel. 

*System time* is the operating system CPU time due to system calls from the process.

In [ ]:
%%time
plot_times([np_mmm, tf_mmm, tf_mmm_gpu, torch_mmm, torch_mmm_cuda], 
           [1,16,128,256,512,1024,2048,4096,8192], repeats=3)

**Exercise**: Why is the user time greater than wall clock time? Explain.

# Links

*   [Using Pytorch and Cuda for Large Computation in Google Colabs](https://medium.com/analytics-vidhya/using-pytorch-and-cuda-for-large-computation-in-google-colabs-f1c026c17673)

